In [ ]:
# biblioca para conectar-se ao postgres
import psycopg2

In [ ]:
# String de conexão
con = psycopg2.connect(host='database-1.ch6ro0aeu2xx.us-east-1.rds.amazonaws.com',
                       database='postgres',
                       user='postgres',
                       password='12345678'
                       )
# transacoes seja autocommitada automaticamente
con.autocommit = True

# cursor é um objeto para executar consulta no banco de dados
cur = con.cursor()

# Instrução que será executada
cur.execute('CREATE DATABASE inventario;')


# fechando a conexão
con.close()

In [ ]:
# String de conexão
con = psycopg2.connect(host='database-1.ch6ro0aeu2xx.us-east-1.rds.amazonaws.com',
                       database='inventario',
                       user='postgres',
                       password='12345678'
                       )
# transacoes seja autocommitada automaticamente
con.autocommit = True

# cursor é um objeto para executar consulta no banco de dados
cur = con.cursor()

# Instrução que será executada
cur.execute('CREATE TABLE arquivos (idarquivo INT, nomearquivo VARCHAR(256));')


# fechando a conexão
con.close()

### Conectando no bucket S3

In [ ]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 7.5 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 9.2 MB 53.0 MB/s 
     |████████████████████████████████| 140 kB 68.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


In [ ]:
import boto3
import io # leitura de arquivos
import psycopg2

In [ ]:
# Interagindo com o bucket S3
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = '',
    aws_secret_access_key = '',
    aws_session_token = ''
)

In [ ]:
# https://prnt.sc/mReAXjm_7NvQ
bucket = 'imagensengdados3210'
prefix = 'imagens/'

### Inserindo dados na tabela

In [ ]:
con = psycopg2.connect(host='database-1.ch6ro0aeu2xx.us-east-1.rds.amazonaws.com',
                       database='inventario',
                       user='postgres',
                       password='12345678'
                       )
con.autocommit = True
cur = con.cursor()
id = 0

# Percorre todos os objetos do prefix
for objects_s3 in s3.Bucket(bucket).objects.filter(Prefix=prefix):

  # verifica se o arquivo termina com jpg
  if objects_s3.key.endswith('jpg') or objects_s3.key.endswith('JPG'):

    # armazena a imagem jpg na variavel 'filename'
    # remove o 'imagens/' da url da imagem 'imagens/avenue-g9ad8b9b60_640.jpg'
    filename = objects_s3.key.split('/')[1]
    id+=1
    cur.execute("INSERT INTO arquivos (idarquivo, nomearquivo) VALUES ("+str(id)+",'"+filename+"')")
        

### Verificando se os nomes dos arquivos foram inseridos no banco

In [ ]:
# String de conexão
con = psycopg2.connect(host='database-1.ch6ro0aeu2xx.us-east-1.rds.amazonaws.com',
                       database='inventario',
                       user='postgres',
                       password='12345678'
                       )
# transacoes seja autocommitada automaticamente
con.autocommit = True

# cursor é um objeto para executar consulta no banco de dados
cur = con.cursor()

# Instrução que será executada
cur.execute('SELECT * FROM arquivos;')

# variável que retorna todos os dados da tabela
recset = cur.fetchall

# percorre cada dado da variavel recset
for rec in recset():
  print(rec)

# fechando a conexão
con.close()

(1, 'avenue-g9ad8b9b60_640.jpg')
(2, 'dandelion-ge4c90edd8_640.jpg')
(3, 'fantasy-g95f970a56_640.jpg')
(4, 'garden-g42e486784_640.jpg')
(5, 'pink-ge82d54651_640.jpg')
(6, 'road-g37132565b_640.jpg')
(7, 'road-g41ea28d46_640.jpg')
(8, 'sunset-gac16749a1_640.jpg')
(9, 'tree-g386d6021c_640.jpg')
(10, 'tree-gd34ff2fcb_640.jpg')
